# Obteniendo el TOTAL de casos positivos, fallecidos y vacunados contra COVID-19 por cada departamento del Perú

En este notebook se busca obtener un dataset con el TOTAL con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** por cada **departamento** del Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

En esta sección se lee y procesa el dataset **Data Product 1** ([DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)) para obtener un resumen general por departamento.

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
# Cargamos un diccionario con el número de habitantes por departamento en 2020
dic_dep = {
    "AMAZONAS"	:   426806,
    "ANCASH"	:   1180638,
    "APURIMAC"	:   430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:   668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
}

# Convertimos a dataframe
df_dep = pd.DataFrame(dic_dep.items(), columns = ['departamento', 'no_habitantes'])

In [3]:
# Leer el DP1
url_dp1 = '../Data/DP1_covid19-peru_x_semanaEpi.csv'
df = pd.read_csv(url_dp1)

# Procesar el df (sumar cada col)
df = pd.DataFrame(df.sum(axis = 0, skipna = True)).reset_index().rename({'index': 'var', 0: 'total'}, axis=1)

# Dejar solamente filas que contengan el nombre de un departamento
lst_dfs = [] 

for dep in df_dep['departamento']:
    temp_df = df.loc[df['var'].str.contains(dep)]
    if not temp_df.empty:
        lst_dfs.append(temp_df)
        
df = pd.concat(lst_dfs, ignore_index=True).drop_duplicates()  # Drop duplicates

# Eliminamos a vacunados (_vac) porque necesitamos todas las dosis 1, 2 y 3 no solo vacunación completa
df = df[~df['var'].str.contains('_vac')]  
    
# Separate in sub dfs and remove substrings
df_cas = df[df['var'].str.contains('_cas')].rename({'var': 'departamento','total': 'casos'}, axis = 1)
df_cas['departamento'] = df_cas['departamento'].str.replace('_cas', '')
df_fal = df[df['var'].str.contains('_fal')].rename({'var': 'departamento','total': 'fallecidos'}, axis = 1)
df_fal['departamento'] = df_fal['departamento'].str.replace('_fal', '')

# Unir en un solo dataframe el resumen por departamentos 
df_summary = df_dep.merge(df_cas, how='left', on='departamento')
df_summary = df_summary.merge(df_fal, how='left', on='departamento')

df_summary

,departamento,no_habitantes,casos,fallecidos
0,AMAZONAS,426806,36008,1294
1,ANCASH,1180638,98994,6807
2,APURIMAC,430736,30020,1537
3,AREQUIPA,1497438,151566,9793
4,AYACUCHO,668213,40342,2178
5,CAJAMARCA,1453711,80198,4226
6,CALLAO,1129854,124212,10186
7,CUSCO,1357075,99157,4874
8,HUANCAVELICA,365317,19153,1190
9,HUANUCO,760267,42930,2747


## B. Agregar el número de dosis

In [4]:
# Leer solamente "id_eess" y "dosis" del dataset de vacunados
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['id_eess', 'dosis']
df_vac = dd.read_csv(url_vac, sep = ",", usecols = vac_col, dtype = {'id_eess':'int32','dosis': 'category'})

# Obtener las variables dummies de la variable "dosis"
df_vac = df_vac.categorize()
dosis = dd.get_dummies(df_vac['dosis']).drop(['4', '5'], axis = 1)

# Ahora se tiene la ubicación y el número de dosis aplicada
df_vac = dd.concat([df_vac, dosis], axis=1, ignore_unknown_divisions=True)
del dosis, df_vac['dosis']

In [5]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = '../RawData/TB_UBIGEOS.csv'
url_vaccenter = '../RawData/TB_EESS.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante "id_ubigeo"
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

In [6]:
# Encontrar el departamento correspondiente de todas las personas vacunadas
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')

# Contar el total de número de dosis por departamento
df_vac = df_vac.groupby('departamento').agg({'1': ['sum'], '2': ['sum'], '3': ['sum']}).compute()
df_vac = df_vac.astype({('1', 'sum'): 'int64', ('2', 'sum'): 'int64', ('3', 'sum'): 'int64'})

In [7]:
# Unir al df resumen (df_summary)
df_summary = df_summary.merge(df_vac, how='left', on='departamento')

# Rename cols
df_summary.rename(columns = {'index':'departamentos', ('1', 'sum'):'dosis_1', 
                             ('2', 'sum'): 'dosis_2', ('3', 'sum'):'dosis_3'}, inplace = True)

df_summary

/home/xxotto/anaconda3/envs/emergelab/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,426806,36008,1294,269759,234881,62185
1,ANCASH,1180638,98994,6807,925975,874008,308021
2,APURIMAC,430736,30020,1537,349508,309755,79731
3,AREQUIPA,1497438,151566,9793,1178803,1086053,333197
4,AYACUCHO,668213,40342,2178,442797,366567,62548
5,CAJAMARCA,1453711,80198,4226,1053456,929044,175825
6,CALLAO,1129854,124212,10186,893294,854007,364952
7,CUSCO,1357075,99157,4874,1001625,882695,181502
8,HUANCAVELICA,365317,19153,1190,249580,202292,36239
9,HUANUCO,760267,42930,2747,533940,454859,101605


## C. Añadir detalles

In [8]:
# Mortality rate per 100k per department
df_summary['tasa_mortalidad'] = round((df_summary['fallecidos']/df_summary['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
df_summary['vac_porcentaje'] = round((df_summary['dosis_2']*100)/df_summary['no_habitantes'],2)

df_summary

,departamento,no_habitantes,casos,fallecidos,dosis_1,dosis_2,dosis_3,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,426806,36008,1294,269759,234881,62185,303.18,55.03
1,ANCASH,1180638,98994,6807,925975,874008,308021,576.55,74.03
2,APURIMAC,430736,30020,1537,349508,309755,79731,356.83,71.91
3,AREQUIPA,1497438,151566,9793,1178803,1086053,333197,653.98,72.53
4,AYACUCHO,668213,40342,2178,442797,366567,62548,325.94,54.86
5,CAJAMARCA,1453711,80198,4226,1053456,929044,175825,290.70,63.91
6,CALLAO,1129854,124212,10186,893294,854007,364952,901.53,75.59
7,CUSCO,1357075,99157,4874,1001625,882695,181502,359.15,65.04
8,HUANCAVELICA,365317,19153,1190,249580,202292,36239,325.74,55.37
9,HUANUCO,760267,42930,2747,533940,454859,101605,361.32,59.83


In [9]:
# Resumen general
print(f'Total habitantes: {df_summary.no_habitantes.sum()}')
print(f'Total fallecidos: {df_summary.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {df_summary.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {df_summary.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {df_summary.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(df_summary.dosis_2.sum()*100/df_summary.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 203994
Total vacunados con 1 dosis: 24890061
Total vacunados con 2 dosis: 22514663
Total vacunados con 3 dosis: 6782828
Porcentaje de la población vacunada: 69.01


## D. Guardar el dataset

In [10]:
df_summary.to_csv('../Data/DP2_covid19-peru_resumen_x_departamentos.csv',index = False)